In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Train

In [9]:
#
import os

import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader
import numpy as np
import tqdm
from datasets import train_dataset, test_dataset
from params_sssd import params
from SSSD import SSSD, align, loss_fn, eval_error

lr = 1e-2
batch_size = 256
epoch = 50

device = torch.device('cuda:2')

model = SSSD(params=params, mode=params.mode, measure=params.measure, bidirectional=params.bidirectional, transpose=params.transpose).to(device)
try:
    model.load_state_dict(torch.load(r'/home/wyl/projects/_BSS_hijack/_end_to_end_compare/02_SSSD/ckpt_sssd/ckpt.pth'))
    print('model loaded')
except:
    print('model does not exise')

train_data = train_dataset()
test_data = test_dataset()
data_loader = DataLoader(train_data, batch_size=16, shuffle=True)
test_loader = DataLoader(test_data, batch_size=16, shuffle=True)

optimizer = AdamW(model.parameters(), lr=lr, eps=1e-6)

tqdm_epoch = tqdm.notebook.tqdm(range(epoch))

for _ in tqdm_epoch:
    avg_loss = 0.
    num_items = 0
    for train_data, truth in data_loader:
        train_data = align(train_data.squeeze(1)).to(device)
        truth = align(truth.squeeze(1)).to(device)

        output = model(train_data)
        loss = loss_fn(output, truth)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        avg_loss += loss.item() * train_data.shape[0]
        num_items += train_data.shape[0]

    tqdm_epoch.set_description('Average Loss:{:5f}'.format(avg_loss / num_items))
    torch.save(model.state_dict(), r'/home/wyl/projects/_BSS_hijack/_end_to_end_compare/02_SSSD/ckpt_sssd/ckpt.pth')


    with torch.no_grad():
        
        test_loss = 0.
        test_items = 0
        for test_data, test_truth in test_loader:
            test_data = align(test_data.squeeze(1)).to(device)
            test_truth = align(test_truth.squeeze(1)).to(device)

            output = model(test_data)
            loss = loss_fn(output, test_truth)

            test_loss += loss.item() * train_data.shape[0]
            test_items += train_data.shape[0]

    # print('test loss: {:5f}'.format(test_loss / test_items))
    # Print the averaged training loss so far.
    tqdm_epoch.set_description('Average Loss: {:5f},   Test Loss: {:5f}'.format(avg_loss / num_items, test_loss / test_items))
    # Update the checkpoint after each epoch of training.
    torch.save(model.state_dict(), '/home/wyl/projects/_BSS_latent/ckpt/ckpt.pth') 

model does not exise


  0%|          | 0/50 [00:00<?, ?it/s]

[KeOps] Generating code for formula Sum_Reduction(ComplexMult(Real2Complex(1/ComplexSquareAbs(ComplexMult(Var(1,2,0)-Var(2,2,1),Var(1,2,0)-Conj(Var(2,2,1))))),ComplexMult(Var(1,2,0)*ComplexReal(Var(0,2,1))-Real2Complex(Sum(Var(0,2,1)*Var(2,2,1))),Conj(ComplexMult(Var(1,2,0)-Var(2,2,1),Var(1,2,0)-Conj(Var(2,2,1)))))),0) ... OK
[KeOps] Generating code for formula Sum_Reduction(Real2Complex(ComplexMult(Conj(Conj(ComplexMult(Var(1,2,0)-Var(2,2,1),Var(1,2,0)-Conj(Var(2,2,1))))),ComplexMult(Conj(Real2Complex(1/ComplexSquareAbs(ComplexMult(Var(1,2,0)-Var(2,2,1),Var(1,2,0)-Conj(Var(2,2,1)))))),Var(3,2,0)))|Var(1,2,0))-Var(2,2,1)*ComplexReal(ComplexMult(Conj(Conj(ComplexMult(Var(1,2,0)-Var(2,2,1),Var(1,2,0)-Conj(Var(2,2,1))))),ComplexMult(Conj(Real2Complex(1/ComplexSquareAbs(ComplexMult(Var(1,2,0)-Var(2,2,1),Var(1,2,0)-Conj(Var(2,2,1)))))),Var(3,2,0)))),1) ... OK
[KeOps] Generating code for formula Sum_Reduction(((-ComplexMult(Conj(Var(1,2,0)-Conj(Var(2,2,1))),ComplexMult(Conj(Conj(ComplexMult(